In [1]:
# %pip install pandas
# %pip install gingerit
# %pip install regex
# %pip install openpyxl

In [2]:
import pandas as pd
import regex as re
from gingerit.gingerit import GingerIt
parser = GingerIt()

In [3]:
ig_df = pd.read_csv("instagram_posts.csv")
ig_df.head()

,id,post_url,is_video,user_name,caption
0,3.122840e+18,https://scontent-ber1-1.cdninstagram.com/v/t51...,False,memezar,Swipe ⬅️ Random Meme Dump # 🥈0️⃣\n\nLike this ...
1,3.069080e+18,https://scontent-ber1-1.cdninstagram.com/v/t51...,False,memezar,Y’all gonna miss these when TikTok is gone⁉️🤣\...
2,3.075080e+18,https://scontent-ber1-1.cdninstagram.com/v/t51...,False,memezar,I want to love my job like this guy 😂👏\n\nLike...
3,3.149520e+18,https://scontent-ber1-1.cdninstagram.com/v/t51...,False,memezar,Grandpa living in the moment 😂🙌\n\nLike this c...
4,3.149050e+18,https://scontent-ber1-1.cdninstagram.com/v/t51...,False,memezar,Swipe ⬅️ Random Meme Dump #108 😂\n\nLike this ...


In [4]:
def remove_emoticons_hashtags_tags(text):
    # Remove emoticons
    text = re.sub(r'[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F700-\U0001F77F\U0001F800-\U0001F8FF\U0001F900-\U0001F9FF\U0001FA00-\U0001FA6F\U0001FA70-\U0001FAFF\U00002702-\U000027B0\U000024C2-\U0001F251]+', '', text)

    # Remove hashtags and @tags
#     text = re.sub(r'[@#]\w+', '', text)
# The pattern [#@]\S+\s will find words that start with @ or # and continue until a space is found in the given text. 
#  It will capture the entire word along with any special characters until the space is reached.
    text +=" "
    text = re.sub(r'[#@]\S+\s', '', text)

    # Remove anything inside parentheses
    text = re.sub(r'\([^)]*\)', '', text)
    
    # Remove any trailing punctuations
    text = text.replace("@","").replace("#","")
    
    return text.strip()


gingered_captions = []
cleaneded_captions = []
Error_status = []

def check_correction_type(correction):
    excluded_correction_types = ['Accept space','Accept comma addition']
    for correction in ginger_cap['corrections']:
        print("\t ------------------",correction['definition'])
        if correction['definition'] in excluded_correction_types:
            return False
    return True

for i,caption in enumerate(ig_df.caption):
    try:
        clean_caption = remove_emoticons_hashtags_tags(caption)
        ginger_cap = parser.parse(clean_caption)
        error_status = len(ginger_cap['corrections']) > 0
        
        # append to lists
        cleaneded_captions.append(clean_caption)
        gingered_captions.append(ginger_cap)
        
        # logic to ignore few types of corrections
        if error_status:
            for correction in ginger_cap['corrections']:
                new_error_status=check_correction_type(correction)
            Error_status.append(new_error_status)
            print(i,len(caption),new_error_status)
        else:
            print(i,len(caption),"False")
            Error_status.append(False)
    except Exception as Err:
        print(Err,i, "failed!")
        
        # append to lists
        cleaneded_captions.append("FAILED")
        gingered_captions.append("FAILED")
        Error_status.append(False)

0 175 False
1 254 False
2 193 False
3 158 False
4 161 False
5 218 False
6 137 False
7 169 False
8 162 False
9 181 False
10 108 False
11 169 False
	 ------------------ Accept comma addition
	 ------------------ Accept comma addition
	 ------------------ Accept comma addition
12 516 False
13 25 False
14 190 False
	 ------------------ Accept space
15 210 False
16 257 False
	 ------------------ a natural and periodic state of rest during which consciousness of the world is suspended
	 ------------------ Accept comma addition
	 ------------------ a natural and periodic state of rest during which consciousness of the world is suspended
	 ------------------ Accept comma addition
17 370 False
	 ------------------ None
18 284 True
19 299 False
20 274 False
	 ------------------ Accept space
21 359 False
22 240 False
23 506 False
	 ------------------ Accept space
	 ------------------ Accept space
	 ------------------ Accept space
24 590 False
'Corrections' 25 failed!
	 ------------------ None
	 -

In [5]:
ig_df['cleaneded_captions']=cleaneded_captions
ig_df['gingered_captions']=gingered_captions
ig_df['error_status']=Error_status

In [6]:
ig_df.to_excel("gingered_posts2.xlsx")

----------

# TESTING

In [10]:
text = """Gamers or husters. Corprate wariors or students. Whatever be their trip, what trips them up is a laptop that just cannot keep pace. #NewWork for @reliance_digital as part of “The Hunt for India’s Slowest Laptop” campaign by Team 21N.

Credets:
Brand: @reliance_digital
Client: Manoj Jain, Nisha Kallianpur, Anamika Tiwari, Esha Nath
Production House: @firsttakefilms
Direction: @sagnik6285
Creative Team: @bingeljell, @inde.stagram, @_kashishkakkar_
Account Management: @neerajkarkera , @divya_jain_dj, @aarsha_rathi
Founder & CEO: @the.malayali"""

parser.parse(remove_emoticons_hashtags_tags(text))

{'text': 'Gamers or husters. Corprate wariors or students. Whatever be their trip, what trips them up is a laptop that just cannot keep pace. for as part of “The Hunt for India’s Slowest Laptop” campaign by Team 21N.\n\nCredets:\nBrand: Client: Manoj Jain, Nisha Kallianpur, Anamika Tiwari, Esha Nath\nProduction House: Direction: Creative Team: Account Management: , Founder & CEO:',
 'result': 'Gamers or holsters. Corporate workers or students. Whatever be their trip, what trips them up is a laptop that just cannot keep pace. Four as part of “The Hunt for India’s Slowest Laptop” campaign by Team 21N.\n\nCredits:\nBrand: Client: Manoj Jain, Nisha Kallianpur, Anamika Tiwari, Esha Nath\nProduction House: Direction: Creative Team: Account Management:, Founder & CEO:',
 'corrections': [{'start': 351,
   'text': ' ,',
   'correct': ',',
   'definition': 'Accept space'},
  {'start': 208, 'text': 'Credets', 'correct': 'Credits', 'definition': None},
  {'start': 132, 'text': 'for', 'correct': 'F